In [1]:
import numpy as np

Source is from here: http://mlwiki.org/index.php/Gram-Schmidt_Process#QR_Factorization Approach is similar to the one used in gram-schmidt orthogonization.

In [2]:
def qr_factorization(A):
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))

    for j in range(n):
        v = A[:, j]

        for i in range(j - 1):
            q = Q[:, i]
            R[i, j] = q @ v
            v = v - R[i, j] * q

        norm = np.linalg.norm(v)
        Q[:, j] = v / norm
        R[j, j] = norm
    return Q, R

Example usage

In [3]:
np.random.seed(1)
# A = np.random.rand(13, 10) * 1000
A = np.array([[60, 91, 26], [60, 3, 75], [45, 90, 31]], dtype='float')
Q, R = qr_factorization(A)
Q.shape, R.shape

((3, 3), (3, 3))

In [4]:
Q

array([[ 0.62469505,  0.71080736, -0.63928383],
       [ 0.62469505,  0.02343321,  0.75368929],
       [ 0.46852129,  0.70299629, -0.15254061]])

In [5]:
R

array([[ 96.04686356,   0.        ,  77.61835966],
       [  0.        , 128.02343535,   0.        ],
       [  0.        ,   0.        ,  35.17655816]])

In [6]:
np.allclose(A, Q@R)

True

In [7]:
np.allclose(np.abs((Q, R)), np.abs(np.linalg.qr(A)))

False

In [8]:
np.linalg.qr(A)

(array([[-0.62469505,  0.35495981, -0.69552831],
        [-0.62469505, -0.76160078,  0.17239591],
        [-0.46852129,  0.54218796,  0.69750987]]),
 array([[ -96.04686356, -100.88825018,  -77.61835966],
        [   0.        ,   78.81345682,  -31.08327672],
        [   0.        ,    0.        ,   16.46876292]]))

In [9]:
np.abs((A - Q @ R).sum()) < 1e-6

True

Second version using Householder Transformation

In [10]:
def qr_householder(A):
    m, n = A.shape
    Q = np.eye(m) # Orthogonal transform so far
    R = A.copy() # Transformed matrix so far

    for j in range(n):
        # Find H = I - beta*u*u' to put zeros below R[j,j]
        x = R[j:, j]
        normx = np.linalg.norm(x)
        rho = -np.sign(x[0])
        u1 = x[0] - rho * normx
        u = x / u1
        u[0] = 1
        beta = -rho * u1 / normx

        R[j:, :] = R[j:, :] - beta * np.outer(u, u) @ R[j:, :]
        Q[:, j:] = Q[:, j:] - beta * Q[:, j:] @ np.outer(u, u)
        
    return Q, R

In [11]:
Q_h, R_h = qr_householder(A)
Q_h.shape, R_h.shape

((3, 3), (3, 3))

In [12]:
np.abs((A - Q_h @ R_h).sum()) < 1e-6

True

In [13]:
np.sqrt((12*12)+(6*6)+(4*4))

14.0

In [14]:
((2*2)+(6*6)+(4*4))

56

In [15]:
(1)+(3*3)+(2*2)

14

In [16]:
Q_h

array([[-0.62469505,  0.35495981,  0.69552831],
       [-0.62469505, -0.76160078, -0.17239591],
       [-0.46852129,  0.54218796, -0.69750987]])

In [17]:
R_h

array([[ -96.04686356, -100.88825018,  -77.61835966],
       [   0.        ,   78.81345682,  -31.08327672],
       [   0.        ,    0.        ,  -16.46876292]])

In [18]:
import numpy as np
from typing import Union
#source https://stackoverflow.com/questions/53489237/how-can-you-implement-householder-based-qr-decomposition-in-python

def householder(x: np.ndarray) -> Union[np.ndarray, int]:
    alpha = x[0]
    s = np.power(np.linalg.norm(x[1:]), 2)
    v = x.copy()

    if s == 0:
        tau = 0
    else:
        t = np.sqrt(alpha**2 + s)
        v[0] = alpha - t if alpha <= 0 else -s / (alpha + t)

        tau = 2 * v[0]**2 / (s + v[0]**2)
        v /= v[0]

    return v, tau

def householder_vectorized(a):
    """Use this version of householder to reproduce the output of np.linalg.qr 
    exactly (specifically, to match the sign convention it uses)

    based on https://rosettacode.org/wiki/QR_decomposition#Python
    """
    v = a / (a[0] + np.copysign(np.linalg.norm(a), a[0]))
    v[0] = 1
    tau = 2 / (v.T @ v)

    return v,tau

def qr_decomposition(A: np.ndarray):
    m,n = A.shape
    R = A.copy()
    Q = np.identity(m)

    for j in range(0, n):
        # Apply Householder transformation.
        v, tau = householder(R[j:, j])
        print(v)
        H = np.identity(m)
        H[j:, j:] -= tau * v.reshape(-1, 1) @ v
        R = H @ R
        Q = H @ Q

    return Q[:n].T, R[:n]

In [19]:
A

array([[60., 91., 26.],
       [60.,  3., 75.],
       [45., 90., 31.]])

In [21]:
# qr_decomposition(A)

In [24]:
m = 5
n = 4
A = np.random.rand(m, n)
q, r = np.linalg.qr(A)
# Q, R = qr_decomposition(A)

In [29]:
import numpy as np
#https://rosettacode.org/wiki/QR_decomposition#Python
def qr(A):
    m, n = A.shape
    Q = np.eye(m)
    for i in range(n - (m == n)):
        H = np.eye(m)
        H[i:, i:] = make_householder(A[i:, i])
        print(H)
        Q = np.dot(Q, H)
        A = np.dot(H, A)
    return Q, A
 
def make_householder(a):
    v = a / (a[0] + np.copysign(np.linalg.norm(a), a[0]))
    v[0] = 1
    H = np.eye(a.shape[0])
    H -= (2 / np.dot(v, v)) * np.dot(v[:, None], v[None, :])
    return H
 
# task 1: show qr decomp of wp example
a = np.array(((
    (12, -51,   4),
    ( 6, 167, -68),
    (-4,  24, -41),
)))
 
q, r = qr(a)

[[-0.85714286 -0.42857143  0.28571429]
 [-0.42857143  0.9010989   0.06593407]
 [ 0.28571429  0.06593407  0.95604396]]
[[ 1.          0.          0.        ]
 [ 0.         -0.99384615 -0.11076923]
 [ 0.         -0.11076923  0.99384615]]


In [26]:
q.round(4)

array([[-0.8571,  0.3943,  0.3314],
       [-0.4286, -0.9029, -0.0343],
       [ 0.2857, -0.1714,  0.9429]])

In [27]:
r.round(4)

array([[ -14.,  -21.,   14.],
       [   0., -175.,   70.],
       [   0.,   -0.,  -35.]])

In [ ]:
np.linalg.qr(a)